# behavior distribution plot

In [ ]:
import EEGAnalysis as ea
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [ ]:
dm = ea.DataManager('/media/STORAGE/EEG/Data')

patient_id = 'wangjinan'
patient = dm.get_patient(patient_id)

_bdf = patient.get_marker(marker='behavior')
_mdf = patient.get_marker(marker='marker')

In [ ]:
_result = []
for item in _bdf.file.unique():
    _marker = _mdf['marker'][_mdf['file'] == item].values
    _behavior = _bdf['marker'][_bdf['file'] == item].values
    _q = ea.get_relative_behavior_time(_marker, _behavior)
    _result.append(_q)

In [ ]:
# In this code, [:,1,:], 1 was 5-s interval; 0 was 10-s interval
_result_par = np.array(_result).reshape(-1, 2, 20)[:,1,:] 

plt.figure(figsize=(25,5))
for i in range(5):
    plt.subplot(1,5,i+1)
    behavior_stat(_result_par[:,i*4:(i+1)*4])

plt.suptitle(patient_id)
plt.show()

---

In [ ]:
def behavior_stat(btime, dw=3):
    x = []
    y = []
    for each in btime:
        each = each[~np.isnan(each)]
        x += list(each[:-1])
        y += list(each[1:])
    
    x = np.array(x)
    y = np.array(y)
    
#     _del = ~((x > 0) & (y > 0))
#     x = x[_del]
#     y = y[_del]

    a = stats.gaussian_kde(np.vstack((x,y)))

    xx, yy = np.mgrid[-3:3:0.01, -3:3:0.01]
    positions = np.vstack([xx.ravel(), yy.ravel()])

    plt.contourf(xx, yy, a(positions).reshape(xx.shape), 80, cmap=plt.get_cmap('rainbow'))
    plt.hlines([0], -3, 3, alpha=0.5)
    plt.vlines([0], -3, 3, alpha=0.5)
    plt.plot([-4,4],[-4,4], 'k', alpha=0.5)
    plt.scatter(x,y, c='k', alpha=1)

    plt.xlim((-dw,dw-0.01))
    plt.ylim((-dw,dw-0.01))

    plt.xlabel('prior tapping')
    plt.ylabel('latter tapping')
    return x, y